In [ ]:
import feedparser
import pandas as pd

# RSS URL
rss_url = "http://kind.krx.co.kr:80/disclosure/rsstodaydistribute.do?method=searchRssTodayDistribute&repIsuSrtCd=&mktTpCd=0&searchCorpName=&currentPageSize=500"
# http://kind.krx.co.kr:80/disclosure/rsstodaydistribute.do?method=searchRssTodayDistribute&repIsuSrtCd=&mktTpCd=0&searchCorpName=&currentPageSize=15
# http://kind.krx.co.kr:80/disclosure/rsstodaydistribute.do?method=searchRssTodayDistribute&repIsuSrtCd=&mktTpCd=0&searchCorpName=&currentPageSize=15
# RSS 데이터 가져오기
feed = feedparser.parse(rss_url)

# RSS 항목에서 필요한 데이터 추출
data = []
for entry in feed.entries:
    data.append({
        "title": entry.title,
        "link": entry.link,
        "published": entry.published,
        # "summary": entry.summary if hasattr(entry, 'summary') else None
    })

# 판다스 데이터프레임 생성
df = pd.DataFrame(data)

# 데이터프레임 출력
df
# 필요하면 엑셀로 저장
# df.to_excel("krx_rss_data.xlsx", index=False, encoding="utf-8-sig")


title  \
0               [코]에스유홀딩스 [정정]전환사채권발행결정(33회차)   
1                   유가증권시장 상장지수증권 유동성공급자 보유수량   
2              [코]에스유홀딩스 [정정]전환사채권발행결정(제32회차)   
3                     코스닥시장 자기주식매매신청내역(코스닥시장)   
4    코스닥시장 직전1개월 일평균대비 거래량 증가율 상위 10종목(코스닥시장)   
..                                        ...   
423                          [코]SK브로드밴드 투자설명서   
424                   [유]일성건설 [정정]단일판매ㆍ공급계약체결   
425               [유]삼성전자 임원ㆍ주요주주특정증권등소유상황보고서   
426            [유]한미사이언스 풍문 또는 보도에 대한 해명(미확정)   
427                     [코]미디어젠 횡령ㆍ배임 혐의 진행사항   

                                                  link  \
0    http://kind.krx.co.kr:80/common/disclsviewer.d...   
1    http://kind.krx.co.kr:80/common/disclsviewer.d...   
2    http://kind.krx.co.kr:80/common/disclsviewer.d...   
3    http://kind.krx.co.kr:80/common/disclsviewer.d...   
4    http://kind.krx.co.kr:80/common/disclsviewer.d...   
..                                                 ...   
423  http://kind.krx.co.kr:80/common/disclsviewer.d...   
424  http://kind.krx.co.kr:80/common/disclsviewer.d...   
425  http://kind.krx.co.kr:80/common/disclsviewer.d...   
426  http://kind.krx.co.kr:80/common/disclsviewer.d...   
427  http://kind.krx.co.kr:80/common/disclsviewer.d...   

                           published  
0    Wed, 04 Dec 2024 18:31:00 +0900  
1    Wed, 04 Dec 2024 18:31:00 +0900  
2    Wed, 04 Dec 2024 18:21:00 +0900  
3    Wed, 04 Dec 2024 18:08:00 +0900  
4    Wed, 04 Dec 2024 18:08:00 +0900  
..                               ...  
423  Wed, 04 Dec 2024 08:24:00 +0900  
424  Wed, 04 Dec 2024 08:03:00 +0900  
425  Wed, 04 Dec 2024 07:55:00 +0900  
426  Wed, 04 Dec 2024 07:46:00 +0900  
427  Wed, 04 Dec 2024 07:41:00 +0900  

[428 rows x 3 columns]

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from io import BytesIO
import pandas as pd
from datetime import datetime,timedelta
import numpy as np

def corp_open_list(date):
    gen_otp_url = 'https://kind.krx.co.kr/disclosure/todaydisclosure.do'
    gen_otp = {
        'method': 'searchTodayDisclosureSub',
        'currentPageSize': '3000',
        'pageIndex': '1',
        'orderMode': '0',
        'orderStat': 'D',
        # 'marketType': ,
        'forward': 'todaydisclosure_sub',
        # 'searchMode': ,
        # 'searchCodeType': ,
        'chose': 'S',
        'todayFlag': 'N',
        # 'repIsuSrtCd': ,
        # 'kosdaqSegment': ,
        'selDate': date,
        # 'searchCorpName': ,
        # 'copyUrl': 'http://kind.krx.co.kr:80/disclosure/rsstodaydistribute.do?method=searchRssTodayDistribute&repIsuSrtCd=&mktTpCd=0&searchCorpName=&currentPageSize=15',
        }

    headers = {
            'Referer':'https://kind.krx.co.kr/disclosure/todaydisclosure.do',
            }

    otp_stk = requests.post(gen_otp_url,gen_otp,headers=headers).text


    html_data = otp_stk

    # BeautifulSoup로 HTML 파싱
    soup = BeautifulSoup(html_data, 'html.parser')

    # 테이블 데이터 추출
    data = []
    table_rows = soup.select('table.list tbody tr')  # 테이블의 tbody 내 모든 행을 선택

    for row in table_rows:
        cols = row.find_all('td')  # 각 열(td)을 추출
        if len(cols) >= 3:  # 필요한 열 개수 확인 (최소 title, link, published가 있다고 가정)
            data.append({
                "corp" : cols[1].text.strip(),  # 제목 추출
                "title": cols[2].text.strip(),  # 제목 추출
                # "link": cols[2].a['href'].strip() if cols[2].a else None,  # 링크 추출
                "published":  date,  # 날짜/시간 추출
                "time":  cols[0].text.strip(),  # 날짜/시간 추출
            })

    # 판다스 데이터프레임으로 변환
    df = pd.DataFrame(data)

    return df

start_date = datetime.strptime('2024-10-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-12-04', '%Y-%m-%d')

ddf = pd.DataFrame()
for i in range((end_date - start_date).days + 1):  # 날짜 차이 + 1 (포함하기 위해)
    current_date = start_date + timedelta(days=i)
    date = current_date.strftime('%Y-%m-%d')
    try:
        dd = corp_open_list(date)
        dd = dd[dd['title'].str.contains('특정계좌')]
        ddf = pd.concat([ddf,dd])
    except:
        print(date)
ddf.to_clipboard()
ddf

2024-10-01
2024-10-03
2024-10-05
2024-10-06
2024-10-09
2024-10-12
2024-10-13
2024-10-19
2024-10-20
2024-10-26
2024-10-27
2024-11-02
2024-11-03
2024-11-09
2024-11-10
2024-11-16
2024-11-17
2024-11-23
2024-11-24
2024-11-30
2024-12-01


corp                    title   published   time
200  대신밸런스제13호스팩  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-02  16:20
250          예스티  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-08  16:08
195        타이거일렉  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-10  16:32
192        듀켐바이오  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-11  16:05
127        듀켐바이오  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-14  16:32
117          제노코  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-16  16:36
150        알티캐스트  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-22  16:25
120        듀켐바이오  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-23  16:21
121         진코스텍  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-23  16:21
141           파두  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-25  16:26
253          소니드  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-30  15:55
238         동신건설  [투자주의]특정계좌(군) 매매관여 과다종목  2024-10-31  16:31
151           휴젤  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-06  16:24
146      인크레더블버즈  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-12  16:29
169        스타플렉스  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-15  16:16
213        누리플렉스  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-18  16:10
214      에프엔에스테크  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-18  16:10
159         브이원텍  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-19  16:05
168       한솔피엔에스  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-19  16:01
234          유엔젤  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-22  15:55
137     에스씨엠생명과학  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-25  16:23
165           신풍  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-25  16:12
235         하이소닉  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-26  16:13
236      코스맥스엔비티  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-26  16:13
165       디엔에이링크  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-27  16:39
167       바이브컴퍼니  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-27  16:39
168         플라즈맵  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-27  16:39
305         고려아연  [투자주의]특정계좌(군) 매매관여 과다종목  2024-11-29  16:00
205         고려아연  [투자주의]특정계좌(군) 매매관여 과다종목  2024-12-02  16:03
249         알에프텍  [투자주의]특정계좌(군) 매매관여 과다종목  2024-12-03  15:59
148       세경하이테크  [투자주의]특정계좌(군) 매매관여 과다종목  2024-12-04  16:28
149        코스텍시스  [투자주의]특정계좌(군) 매매관여 과다종목  2024-12-04  16:28
150         나라셀라  [투자주의]특정계좌(군) 매매관여 과다종목  2024-12-04  16:28